# Google Trend Data

# Basic Forecasting Techniques

In [1]:
#Importing the necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'seaborn'

### Reading the dataset

In [ ]:
data = pd.read_csv('C:/Customers_in_a_Shop.csv',header=None)
data

In [ ]:
data.columns = ['Date','Customers']
data['Date'] = pd.to_datetime(data['Date'],format="%Y-%m")
data = data.set_index('Date')

#Shape of the dataset
data.shape

In [ ]:
data.head()

# Missing Data Handling in Time Series
### We have the following methods for treating missing values in the time series data.
1)	Mean Imputation

2)	Last Observation Carried forward

3)	Linear Interpolation

4)	Seasonal Interpolation


In [ ]:
plt.rcParams['figure.figsize']=(17,5)

plt.plot(data,color='black')
plt.title("Customers in a Shop since 1949")
plt.show()

## Mean Imputation

In [ ]:
plt.rcParams['figure.figsize']=(17,5)
data['Customers_mean'] = data['Customers'].fillna(data['Customers'].mean())
plt.plot(data['Customers_mean'],color='black')
plt.title("Mean Imputation of Missing Values")
plt.show()

## Backward Fill Imputation

In [ ]:
plt.rcParams['figure.figsize']=(17,5)

data['Customers_bfill'] = data['Customers'].bfill()

plt.plot(data['Customers_bfill'],color='black')
plt.title("Last Observation Carried Forward")
plt.show()

## Linear Interpolation

In [ ]:
plt.rcParams['figure.figsize']=(17,5)

data['Customers_linear']=data['Customers'].interpolate(method='linear')

plt.plot(data['Customers_linear'],color='black')
plt.title("Linear Interpolation of Missing Values")
plt.show()

## Seasonal Interpolation

In [ ]:
# lets find the dates where we have missing values
data.index[data['Customers'].isnull()]

In [ ]:
data.tail(15)

In [ ]:
#for example for 1960 take average seasonal of previous date
data['1949-03':'1959-03':12]

In [ ]:
data['1949-03':'1959-03':12].sum()

In [ ]:
data['1949-03':'1959-03':12].shape[0]

In [ ]:
data['1949-03':'1959-03':12].shape[1]

In [ ]:
data.loc['1960-03'].fillna((data['1949-03':'1959-03':12].sum())/data['1949-03':'1959-03':12].shape[0], inplace=True)
data.loc['1954-06'].fillna((data['1949-06':'1953-06':12].sum())/data['1949-06':'1953-06':12].shape[0], inplace=True)
data.loc['1951-07'].fillna((data['1949-07':'1950-07':12].sum())/data.loc['1949-07':'1950-07':12].shape[0], inplace=True)
data.loc['1951-06'].fillna((data['1949-06':'1950-06':12].sum())/data['1949-06':'1950-06':12].shape[0], inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
plt.rcParams['figure.figsize']=(17,5)
plt.plot(data['Customers'],color='black')
plt.title("Seasonal Interpolation of Missing Values")
plt.show()

## Outliers Treatment in Time Series

In [ ]:
plt.rcParams['figure.figsize']=(17,5)
sns.boxplot(data['Customers_linear'], color='brown')

In [ ]:
data['Customers_linear'].sort_values(ascending = False).head(7)

In [ ]:
# outliers treatment

data['Customers_linear'].loc[(data['Customers_linear']>=700)] = 622
# lets also check the null values again
data.isnull().sum()

In [ ]:
plt.rcParams['figure.figsize']=(17,5)
sns.boxplot(data['Customers_linear'], color='brown')
plt.show()

# Normality Analysis

In [ ]:
import seaborn as sns
sns.distplot(data['Customers_linear'])

In [ ]:
import scipy.stats
import pylab 
scipy.stats.probplot(data['Customers_linear'],plot=pylab)
pylab.show()

# Y-Axis: Data Value
# X-Axis: 

* for correct statistic Analysis you can use Zscore function from statsmodels 

# Naive Decomposition
### Additive Seasonal Decomposition

In [ ]:
import statsmodels.api as sm

In [ ]:
plt.rcParams['figure.figsize'] = (17,8)

decomposition = sm.tsa.seasonal_decompose(data['Customers_linear'], model='additive')
decomposition.plot()
                                          
plt.show()

* Be attention at Y-Scale in Seasonal! (-50,+50) from orginal data

### Multiplicative Seasonal Decomposition

In [ ]:
import statsmodels.api as sm
plt.rcParams['figure.figsize'] = (17,8)

decomposition = sm.tsa.seasonal_decompose(data['Customers_linear'], model='multiplicative')
fig = decomposition.plot()
plt.show()

## Splitting Train and Test data

In [ ]:
len(data["Customers_linear"])*0.8

* We Split the data into train and test.
* First 115 rows as the train data and rest other as test data

In [ ]:
data['Customers'] = data['Customers_linear']

In [ ]:
length_train = 115
train = data.iloc[:length_train,:]
train

In [ ]:
test=data.iloc[length_train:,: ]
test

In [ ]:
print(train.shape)
train.tail()

In [ ]:
print(train.shape)
test.head()

# Naive Method
The naive method is the simplest method of all forecasting methods. It looks at the last historical data and extrapolates it for all the future values without adjusting or attempting to establish causal factors.

In [ ]:
train["Customers"][113]

In [ ]:
train["Customers"][114]

In [ ]:
train["Customers"][115]

In [ ]:
y_naive = test.copy()
y_naive['forecasted_naive'] = train.Customers[length_train-1]
y_naive

In [ ]:
plt.rcParams['figure.figsize'] = (17,5)

plt.plot(train['Customers'], label = 'Train')
plt.plot(test['Customers'], label = 'Test')
plt.plot(y_naive['forecasted_naive'], label = 'naive forecast')

plt.legend()
plt.title('Naive Method')
plt.show()

# Simple Average Method
In this method, we take the future predictions equal to the average of all the historical data.

In [ ]:
y_avg = test.copy()

y_avg['forecasted_avg'] = train['Customers'].mean()

In [ ]:
plt.rcParams['figure.figsize'] = (17,5)

plt.plot(train['Customers'], label = 'Train')
plt.plot(test['Customers'], label = 'Test')
plt.plot(y_avg['forecasted_avg'], label = 'simple average forecast')

plt.legend()
plt.title('Simple Average Method')
plt.show()

# Simple Moving Average Method
In this method, we take the future predictions equal to the average of a moving window. A window can be a time period of 3 months, 6 months, 9 months or 1 year.

In [ ]:
y_moving = data.copy()

window = 9
y_moving["moving_average_forecast"] = data['Customers'].rolling(window).mean()
# y_moving['moving_average_forecast'][length_train:] = y_moving['moving_average_forecast'][length_train-1]

In [ ]:
plt.rcParams['figure.figsize'] = (17,5)

plt.plot(train['Customers'], label = 'Train')
plt.plot(test['Customers'], label = 'Test')
plt.plot(y_moving['moving_average_forecast'], label = 'simple moving average forecast')

plt.legend()
plt.title('Simple moving Average Method')
plt.show()

* are you serious about using it for time series modeling?

# Simple Exponential Smoothing


In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

ins2 = SimpleExpSmoothing(train['Customers']).fit(smoothing_level=0.8,optimized=False)
ins_cast2 = ins2.forecast(3).rename('alpha=0.8')


#Plot for alpha = 0.8
ax = train['Customers'].plot(marker='o', color='black', figsize=(12,8), legend=True)
ins_cast2.plot(marker='o', ax=ax, color='red', legend=True)
ins2.fittedvalues.plot(marker='o', ax=ax, color='red')

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

#First Instance
ins1 = SimpleExpSmoothing(train['Customers']).fit(smoothing_level=0.2,optimized=False)
ins_cast1 = ins1.forecast(3).rename('alpha=0.2')

#Second Instance
ins2 = SimpleExpSmoothing(train['Customers']).fit(smoothing_level=0.8,optimized=False)
ins_cast2 = ins2.forecast(3).rename('alpha=0.8')

#Third Instance
ins3 = SimpleExpSmoothing(train['Customers']).fit()
ins_cast3 = ins3.forecast(3).rename('alpha=%s'%ins3.model.params['smoothing_level'])


#After creating model we will visualize the plot
ax = train['Customers'].plot(marker='o', color='black', figsize=(12,8), legend=True)


#Plot for alpha =0.2
ins_cast1.plot(marker='+', ax=ax, color='blue', legend=[True])
ins1.fittedvalues.plot(marker='+', ax=ax, color='blue')

#Plot for alpha = 0.8
ins_cast2.plot(marker='o', ax=ax, color='red', legend=True)
ins2.fittedvalues.plot(marker='o', ax=ax, color='red')

#Plot for alpha=Optimized by statsmodel
ins_cast3.plot(marker='*', ax=ax, color='green', legend=True)
ins3.fittedvalues.plot(marker='*', ax=ax, color='green')

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

model = SimpleExpSmoothing(train['Customers'])
model_fit = model.fit(smoothing_level=1)
# model_fit.params

In [ ]:
y_exp = test.copy()
y_exp['Exponential_forecast'] = model_fit.forecast(24)
y_exp

In [ ]:
plt.rcParams['figure.figsize'] = (17,5)

plt.plot(train['Customers'], label = 'Train')
plt.plot(test['Customers'], label = 'Test')
plt.plot(y_exp['Exponential_forecast'], label = 'simple exponential forecast')

plt.legend()
plt.title('Simple Exponential Method')
plt.show()

* Last day for alpha = 1

# Holt Exponential Smoothing
Holt’s exponential smoothing captures the level and trend of time series in the forecast.

The forecast equation is a function of both level and trend.

y(t+1) = l(t) +b(t) 

Where l(t) is the level component and b(t) is the trend component.

The trend component is calculated as shown

b(t) = β(l(t) - l(t-1)) + (1-β)b(t-1) 

Here beta is the smoothing parameter for trend.


In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

model = ExponentialSmoothing(train['Customers'], seasonal_periods=12, trend='multiplicative')
model_fit = model.fit(smoothing_level=0.2,smoothing_slope=0.04)
model_fit.params
y_holtexponential = test.copy()
y_holtexponential['holtexponential_forecast'] = model_fit.forecast(24)

In [ ]:
plt.rcParams['figure.figsize'] = (17,5)

plt.plot(train['Customers'], label = 'Train')
plt.plot(test['Customers'], label = 'Test')
plt.plot(y_holtexponential['holtexponential_forecast'], label = 'Holts exponential forecast')

plt.legend()
plt.title('Holts exponential Method')
plt.show()

# Holt Winter Exponential Smoothing
This techniques forecasts based on level, trend and seasonality.
The forecast equation for this method includes seasonality.
	
y(t+1) = l(t)+b(t)+s(t+1-m) 
Here m is the number of time a season repeats in a time period.

In [ ]:
model = ExponentialSmoothing(train['Customers'], seasonal_periods=12, trend='multiplicative', seasonal='additive')
model_fit = model.fit(smoothing_level=0.2,smoothing_slope=0.04)
model_fit.params
y_holtwinter = test.copy()
y_holtwinter['holtwinter_forecast'] = model_fit.forecast(12)

In [ ]:
plt.rcParams['figure.figsize'] = (17,5)

plt.plot(train['Customers'], label = 'Train')
plt.plot(test['Customers'], label = 'Test')
plt.plot(y_holtwinter['holtwinter_forecast'], label = 'Holts Winters exponential forecast')

plt.legend()
plt.title('Holts winters exponential Method')
plt.show()

# Stationarity
Stationarity means that the statistical properties of a process generating a time series do not change over time. The statistical properties are Mean, variance and covariance which are same irrespective of the time at which you observe them.

In [ ]:
pd.DataFrame(data.Customers.describe())

# Augmented Dickey-Fuller (ADF) Test for Stationarity
Augmented Dickey Fuller test (ADF Test) is a common statistical test used to test whether a given Time series is stationary or not. It is one of the most commonly used statistical test when it comes to analyzing the stationary of a series.
* p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
* p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(data['Customers'])
result

In [ ]:
import statsmodels.tsa.stattools as sts
sts.adfuller(data['Customers'])

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(data['Customers'])
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
print(f'n_lags: {result[2]}')
print("")

for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')  

* p-value > 5% >>>>> H0 Accepted (Fails to Reject) >>>>> Series is non-stationary

* The mean value is not stationary.
* The variance is fluctating over time.

In [ ]:
import datetime

today = datetime.datetime.today()
print(f"{today:%B %d, %Y}")

# Compare with ACF

In [ ]:
#Loading and plotting acf
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(data['Customers'], ax=plt.gca(), lags=30)
plt.show()

# gca: get current axes

# Kwiatkowski-Phillips-Schmidt-Shin (KPSS) Test
The Kwiatkowski–Phillips–Schmidt–Shin (KPSS) test figures out if a time series is stationary around a mean or linear trend, or is non-stationary due to a unit root. A stationary time series is one where statistical properties — like the mean and variance — are constant over time.

For KPSS test,

The Null Hypothesis : The series is stationary when p-value >0.05

Alternate Hypothesis: The series is not stationary when p-value <= 0.05

In [ ]:
#loading kpss from statsmodel
from statsmodels.tsa.stattools import kpss

result = kpss(data['Customers'])
print(f'KPSS Statistic: {result[0]}')
print(f'p-value: {result[1]}')
print(f'num lags: {result[2]}')
print("")


for key, value in result[3].items():
    print('Critial Values:')
    print(f'   {key}, {value}')  

* p-value < 5% >>>>> H0 Rejected

# Non-Stationary Series to Stationary Series
There are two tools for converting a non-stationary series into a stationary series.

**1) Differencing**

Differencing tool is used to make the mean constant for a time series. That means it removes the trend from the series. 

**2) Transformation**


The procedure is to find the optimal value of lambda between -5 and +5 to minimize the variance of the time series.

## Box Cox Transformation
A Box Cox transformation is a way to transform non-normal dependent variables into a normal shape. Normality is an important assumption for many statistical techniques; if your data isn't normal, applying a Box-Cox means that you are able to run a broader number of tests.

In [ ]:
from scipy.stats import boxcox

data_boxcox = pd.Series(boxcox(data['Customers'],lmbda=0),index=data.index)
plt.plot(data_boxcox, label="After Box Cox Transformation")
plt.legend()
plt.title("Number of Customers Visiting in an Ice Cream Shop since 1950")
plt.show()

In [ ]:
from scipy import stats
fitted_data, fitted_lambda = stats.boxcox(data['Customers'])

# print(fitted_data)
print(fitted_lambda)

In [ ]:
xx  = stats.boxcox(data['Customers'])
xx[1]

### Quick Example

In [ ]:
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

# generate non-normal data (exponential)
original_data = np.random.exponential(size = 1000)

# transform training data & save lambda value
fitted_data, fitted_lambda = stats.boxcox(original_data)

# creating axes to draw plots
fig, ax = plt.subplots(1, 2)

# plotting the original data(non-normal) and fitted data (normal)
sns.distplot(original_data, hist = False, kde = True,kde_kws = {'shade': True, 'linewidth': 2},
             label = "Non-Normal", color ="green", ax = ax[0])

sns.distplot(fitted_data, hist = False, kde = True,kde_kws = {'shade': True, 'linewidth': 2},
             label = "Normal", color ="green", ax = ax[1])

plt.legend(loc = "upper right")

# rescaling the subplots
fig.set_figheight(5)
fig.set_figwidth(10)

print(f"Lambda value used for Transformation: {fitted_lambda}")

## Differencing

Differencing stabilises the mean of a time series by removing changes in the level of a time series, and therefore eliminating (or reducing) trend and seasonality. 

In [ ]:
data_boxcox_difference= pd.Series(data_boxcox - data_boxcox.shift(), index=data.index)
data_boxcox_difference.dropna(inplace=True)

plt.plot(data_boxcox_difference, label="After Box Cox Transformation and Differencing")
plt.legend()
plt.title("Number of Customers Visisting in a Ice Cream Shop since 1950")
plt.show()

## ADF Test

Checking stationary after transformation

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(data_boxcox_difference)
print(f'ADF Statistic: {result[0]}')
print(f'n_lags: {result[1]:.20f}')
print(f'p-value: {result[1]:.20f}')
for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')

## Auto Correleation Function (ACF)
ACF is an (complete) auto-correlation function which gives us values of auto-correlation of any series with its lagged values. 

In [ ]:
#Loading and plotting acf
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(data_boxcox_difference, ax=plt.gca(), lags=30)
plt.show()

# gca: get current axes

1,3,4,5th have significant correlation with future observations.

Model equation would be:

* y = β0 + β1*y(t-1) + β2*y(t-3) + β3*y(t-4) + β4*y(t-5)

* y(t-1), y(t-3), y(t-4) and y(t-5) are the independent variables.


## Partial Auto Correleation Function (PACF)
Partial autocorrelation function (PACF) gives the partial correlation of a stationary time series with its own lagged values, regressed the values of the time series at all shorter lags. It contrasts with the autocorrelation function, which does not control for other lags.

In [ ]:
#Loading and plottin pacf
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(data_boxcox_difference, ax=plt.gca(), lags=30)
plt.show()

In [ ]:
length_train = 115

train_data_boxcox = data_boxcox[:length_train]
test_data_boxcox = data_boxcox[length_train:]

# for NA
train_data_boxcox_difference = data_boxcox_difference[:length_train-1]
test_data_boxcox_difference = data_boxcox_difference[length_train-1:]

# Auto Regressive Model
Regressive model is forecasting the future observations as a **linear regression** of one or more past observations.

* This model has a parameter called “p” which is the lag order

*  is the maximum number of lags that we consider in order to forecast the future observations.

Autoregressive model equation would be


* y(t) = β0 + β1*y(t-1) + β2*y(t-3) + β3*y(t-4) + β4*y(t-5)


In [ ]:
from statsmodels.tsa.arima_model import ARIMA

model_ar = ARIMA(train_data_boxcox_difference, order=(1,0,0))
model_fit = model_ar.fit()
print(model_fit.params)

* Lag = 4 and Regression on 4 Features (Samples)!

## Transform to Original Time Series Data

In [ ]:
y_ar_new = data_boxcox_difference.copy()
y_ar_new

In [ ]:
y_ar_new['ar_forecast_boxcox_difference'] = model_fit.predict(data_boxcox_difference.index.min(),
                                                              data_boxcox_difference.index.max())

y_ar_new['ar_forecast_boxcox'] = y_ar_new['ar_forecast_boxcox_difference'].cumsum()
y_ar_new['ar_forecast_boxcox'] = y_ar_new['ar_forecast_boxcox'].add(data_boxcox[0]) #add data
y_ar_new['ar_forecast'] = np.exp(y_ar_new['ar_forecast_boxcox']) #transform

## Forecasting

In [ ]:
plt.figure(figsize=(17,8))

plt.plot(train['Customers_linear'], label = 'Train')
plt.plot(test['Customers_linear'], label = 'Test')
plt.plot(y_ar_new['ar_forecast'][test.index.min():], label = 'AR model')

plt.legend()
plt.title('Auto regressive model')
plt.show()

* only Trend Detected (Not Seasonal!)

# Moving Average Method
In Moving Average Model, we consider the past forecasted errors to forecast the future values.

The moving average model has a parameter called “q” which is the size of the moving average window over which linear combinations of errors are calculated.

The mathematical equation is:-

y(t) = µ + φ(k)*ε(t-k)

µ is the mean of the series

ε(t-k) is the past forecasted value

φ(k) is the weight associated with error value


In [ ]:
#from statsmodels.tsa.arima_model import ARIMA

model_ma = ARIMA(train_data_boxcox_difference, order=(0,0,7))
model_fit = model_ma.fit()
print(model_fit.params)

In [ ]:
data_boxcox

## Transform to Original Time Series Data

In [ ]:
y_ma_new = data_boxcox_difference.copy()
y_ma_new['ma_forecast_boxcox_difference'] = model_fit.predict(data_boxcox_difference.index.min(),
                                                              data_boxcox_difference.index.max())

y_ma_new['ma_forecast_boxcox'] = y_ma_new['ma_forecast_boxcox_difference'].cumsum()
y_ma_new['ma_forecast_boxcox'] = y_ma_new['ma_forecast_boxcox'].add(data_boxcox[0])
y_ma_new['ma_forecast'] = np.exp(y_ma_new['ma_forecast_boxcox'])

## ForeCasting

In [ ]:
plt.figure(figsize=(17,8))

plt.plot(train['Customers_linear'], label = 'Train')
plt.plot(test['Customers_linear'], label = 'Test')
plt.plot(y_ma_new['ma_forecast'][test.index.min():], label = 'MA model')

plt.legend()
plt.title('Moving Average regressive model')
plt.show()

# Auto Regressive Moving Average Model (ARMA)
ARMA Model combines both AR and MA model.

It takes into account one or more past observations as well as the past errors.

The ARMA model contains two parameters p and q

p is the highest lag in the time series

q is the number of past errors included
  

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

model_arma = ARIMA(train_data_boxcox_difference, order=(1,0,5))
model_fit = model_arma.fit()
print(model_fit.params)

In [ ]:
y_arma_new = data_boxcox_difference.copy()
y_arma_new['arma_forecast_boxcox_difference'] = model_fit.predict(data_boxcox_difference.index.min(),
                                                                  data_boxcox_difference.index.max())
y_arma_new['arma_forecast_boxcox'] = y_arma_new['arma_forecast_boxcox_difference'].cumsum()
y_arma_new['arma_forecast_boxcox'] = y_arma_new['arma_forecast_boxcox'].add(data_boxcox[0])
y_arma_new['arma_forecast'] = np.exp(y_arma_new['arma_forecast_boxcox'])

In [ ]:
plt.figure(figsize=(17,8))

plt.plot(train['Customers_linear'], label = 'Train')
plt.plot(test['Customers_linear'], label = 'Test')
plt.plot(y_arma_new['arma_forecast'][test.index.min():], label = 'ARMA model')

plt.legend()
plt.title('Auto regressive Moving Average model')
plt.show()

# Auto Regressive Integrated Moving Average Model (ARIMA)

It transform the time series using Box Cox and then itself takes care of the differencing and remove the trend from the time series.

We have three parameters to be used:-

p is the highest lag in the model

d is the degree of differencing to make the series stationary

q is the number of past errors terms included

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

model = ARIMA(train_data_boxcox, order=(1,1,5))
model_fit = model.fit()
print(model_fit.params)

In [ ]:
y_arima_new = data_boxcox_difference.copy()
y_arima_new['arima_forecast_boxcox_difference'] = model_fit.predict(data_boxcox_difference.index.min(),
                                                                    data_boxcox_difference.index.max())
y_arima_new['arima_forecast_boxcox'] = y_arima_new['arima_forecast_boxcox_difference'].cumsum()
y_arima_new['arima_forecast_boxcox'] = y_arima_new['arima_forecast_boxcox'].add(data_boxcox[0])
y_arima_new['arima_forecast'] = np.exp(y_arima_new['arima_forecast_boxcox'])

In [ ]:
plt.figure(figsize=(17,8))

plt.plot(train['Customers_linear'], label = 'Train')
plt.plot(test['Customers_linear'], label = 'Test')
plt.plot(y_arima_new['arima_forecast'][test.index.min():], label = 'ARiMA model')

plt.legend()
plt.title('Auto regressive Integrated Moving Average model')
plt.show()

# Seasonal Auto Regressive Integrated Moving Average Model (SARIMA)

SARIMA model brings all the features of ARIMA model along with the seasonality.

The key elements performed in SARIMA are:-

1. The time series is differenced to make it stationary.

2. The SARIMA equation is a linear combination of past observations and past errors.

3. Seasonal differencing is performed on the time series.

4. SARIMA models future seasonality as a linear combination of past seasonality observations and past seasonality errors.


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(train_data_boxcox_difference, order=(1,1,1), seasonal_order=(1,1,1,6))
model_fit = model.fit()
print(model_fit.params)

In [ ]:
y_sarima_new = data_boxcox_difference.copy()
y_sarima_new['sarima_forecast_boxcox_difference'] = model_fit.predict(data_boxcox_difference.index.min(),
                                                                      data_boxcox_difference.index.max())
y_sarima_new['sarima_forecast_boxcox'] = y_sarima_new['sarima_forecast_boxcox_difference'].cumsum()
y_sarima_new['sarima_forecast_boxcox'] = y_sarima_new['sarima_forecast_boxcox'].add(data_boxcox[0])
y_sarima_new['sarima_forecast'] = np.exp(y_sarima_new['sarima_forecast_boxcox'])

In [ ]:
plt.figure(figsize=(17,8))

plt.plot(train['Customers_linear'], label = 'Train')
plt.plot(test['Customers_linear'], label = 'Test')
plt.plot(y_sarima_new['sarima_forecast'][test.index.min():], label = 'SARIMA model')

plt.legend()
plt.title('Seasonal Auto regressive Integrated Moving Average model')
plt.show()